In [1]:
import pandas as pd
import os
import numpy as np
import soundfile as sf
from IPython.display import clear_output
import pickle
import librosa
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import Model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [ ]:
# Wczytuję model
model = tf.keras.models.load_model('/kaggle/input/cnn_mel/keras/default/1/model.h5')

# Wczytuję dane
X_train = np.load('/kaggle/input/data-to-train-mel-spectrogram/X_train.npy')
y_train = np.load('/kaggle/input/data-to-train-mel-spectrogram/y_train.npy')


# Dzielę dane na 5 fragmentów, wykorzstuję 5 kont kaggle do obliczeń aby przyspieszcyć proces
x = len(X_train)
part_size = x // 5

# Podział X_train na 5 części
X_train = [
    X_train[:part_size],                  # Pierwsze 20%
    X_train[part_size:2*part_size],       # Drugie 20%
    X_train[2*part_size:3*part_size],     # Trzecie 20%
    X_train[3*part_size:4*part_size],     # Czwarte 20%
    X_train[4*part_size:]                 # Ostatnie 20%
]

# Podział y_train na 5 części
y_train = [
    y_train[:part_size],                  # Pierwsze 20%
    y_train[part_size:2*part_size],       # Drugie 20%
    y_train[2*part_size:3*part_size],     # Trzecie 20%
    y_train[3*part_size:4*part_size],     # Czwarte 20%
    y_train[4*part_size:]                 # Ostatnie 20%
]

In [ ]:
# Funkcja służy do stworzenia embeddingu nagrania, pobiera po prostu wartości jakie są w wartowie bottleneck podczas dokonywania klasyfikacji danego nagrania
def calcuate_embedding(one_audio, model):

    intermediate_layer_model = Model(inputs=model.inputs,
                                 outputs=model.get_layer('bottleneck').output)
    intermediate_output = intermediate_layer_model.predict(one_audio[np.newaxis, ...])
    
    return intermediate_output

In [ ]:
X_train = X_train[0]
y_train = y_train[0]

# Liczymy embeddingi nagrań treningowych UBM aby stworzyć LDA na ich podstawie
X_train_embedding = []

# Iteracja po każdym elemencie w X_train
for i in range(0, len(X_train)):
    # Obliczenie embeddingu dla danego nagrania w X_train przy użyciu modelu
    train_embedding = calcuate_embedding(X_train[i], model)[0]

    # Dodanie embeddingu do listy X_train_embedding
    X_train_embedding.append(train_embedding)

    # Wyczyszczenie poprzedniego outputu i wyświetlenie postępu przetwarzania
    clear_output(wait=True)
    print(i/len(X_train))  # Wyświetla proporcjonalny postęp jako wartość od 0 do 1

# Zapisanie listy embeddingów X_train_embedding do pliku w formacie pickle
with open("X_train_embedding.pkl", "wb") as file:
    pickle.dump(X_train_embedding, file)